In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def load_params(filename):
    file = open(filename, "r")
    data = str.strip(file.read())
    params = data.split(",")
    return params

In [3]:
def load_np(filename):
    a = np.load(filename)
    return a

In [4]:
pd.options.display.max_columns = None

In [5]:
dir_base = './results/cur'
all_data = []
bad = []
for f in os.listdir(dir_base):
    if "." in f:
        continue

    chtc_num = int(f.split("sults_")[1].split("_0")[0])
    
    params = load_params(dir_base + "/" + f + "/params.txt")
    # fix issue where optimizer is labeled as adam but was actually sgd
    if chtc_num <= 10203974:
        params[7] = 'sgd'
    
    if (chtc_num <= 10177641) and params[17] == 'softplus':
        params[18] = 0 # early tests were w/o fixed seed
    elif (chtc_num > 10177641) and (chtc_num < 10203191) and (params[17] == 'softplus'):
        params[18] = 1 # fixed seed
    elif (chtc_num <= 10204142) and (chtc_num >= 10204070) and (params[17] == 'softplus'):
        params[18] = 0
    elif (chtc_num >= 10203979) and (chtc_num <= 10203992) and (params[17] == 'softplus'):
        params[18] = 1
    count = 0
    for g in os.listdir(dir_base + "/" + f):
        count += 1
        if 'preds' in g:
            dataset = g.split("_preds")[0]
    if count == 1:
        bad.append(f)
        continue

    params.append(chtc_num)
    accuracy_file_name = dataset.split("_t")[0]
    if 'adv' in accuracy_file_name:
        accuracy_file_name = accuracy_file_name.split("_adv")[0] + accuracy_file_name.split("adv")[1]
    accuracy = load_np(dir_base + "/" + f + "/accuracy_" + accuracy_file_name + ".npy")
    test_acc = accuracy[0]
    train_acc = accuracy[1]
    params.append(min(test_acc))
    params.append(sum(test_acc)/len(test_acc))
    params.append(max(test_acc))
    params.append(np.std(test_acc))
    params.append(min(train_acc))
    params.append(sum(train_acc)/len(train_acc))
    params.append(max(train_acc))
    params.append(np.std(train_acc))

    # do this almost all the time, except for adv-only tests
    top1 = load_np(dir_base + "/" + f + "/" + dataset + "_top1const.npy")
    top2 = load_np(dir_base + "/" + f + "/" + dataset + "_top2const.npy")
    top3 = load_np(dir_base + "/" + f + "/" + dataset + "_top3const.npy")
    top4 = load_np(dir_base + "/" + f + "/" + dataset + "_top4const.npy")
    top5 = load_np(dir_base + "/" + f + "/" + dataset + "_top5const.npy")
    gradnorm = load_np(dir_base + "/" + f+ "/" + dataset + "_gradnorm.npy")
    for j in [top1, top2, top3, top4, top5, gradnorm]:
        # params.append(sum(j)/len(j)) # 
        params.append(np.percentile(np.array(j), 50))
        params.append(np.percentile(np.array(j), 25))
        params.append(np.percentile(np.array(j), 75))
    all_data.append(params)
    

In [6]:
cols = ['dataset', 'learning_rate', 'epochs', 'batch_size', 'activation', 'nodes_per_layer', 'num_layers',
       'optimizer', 'target_indices', 'target_vals', 'mod_indices', 'new_vals', 'threshold', 'strategy',
       'loss', 'epsilon', 'lrdecay', "method", 'fixed_seed', 'chtc_num', 'min_acc_test', 'avg_acc_test', 'max_acc_test',
       'stdev_acc_test', 'min_acc_train', 'avg_acc_train', 'max_acc_train', 'stdev_acc_train', 'top1', 
       'top1_25', 'top1_75', 'top2', 'top2_25', 'top2_75', 'top3', 'top3_25', 'top3_75', 'top4', 'top4_25', 
       'top4_75', 'top5', 'top5_25', 'top5_75', 'gradnorm', 'gradnorm_25', 'gradnorm_75']
data = pd.DataFrame(all_data, columns=cols)

In [11]:
data[data.min_acc_train >=0.75].to_csv("accurate_data_1018.csv", index=False)

In [12]:
data[data.min_acc_train < 0.75].to_csv("inacc_data_1018.csv", index=False)

In [10]:
data.to_csv("all_results_1018.csv", index=False)